In [24]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline


In [25]:
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')

In [13]:
model.metabolites.ser__L_p.summary()

In [14]:
# TESTING OUT A LOT

with model:
    print("WT")
    solution = model.optimize()
    print('biomass: ')
    print(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    print('glucose: ')
    print(solution.fluxes.EX_glc__D_e)
    print('serine efflux: ')
    print(solution.fluxes.EX_ser__L_e)
    print("Yield on glucose: ")
    print(solution.fluxes.EX_ser__L_e / (-1*model.reactions.EX_glc__D_e.flux))

print('-------------------------------------------------')
print('')
print('-------------------------------------------------')
    
with model:
    print("ALTERED")
    
    # From FSEOF:
    #model.reactions.PFK_3.bounds = 9.5,1000
    #model.reactions.PGI.bounds = 10,1000
    #model.reactions.PGK.bounds = -19,-19
    #model.reactions.PGL.knock_out()
    #model.reactions.PYK.knock_out()
    #model.reactions.GND.knock_out()
    #model.reactions.G6PDH2r.knock_out()
    #model.reactions.PSERT.bounds = 18,1000
    #model.reactions.HEX1.bounds = 9,1000
    #model.reactions.PSP_L.bounds = 18,1000
    #model.reactions.GLCt2pp.bounds = 9.3,1000
    #model.reactions.Htex.bounds = -19,-19
    #model.reactions.PAPSR2.bounds = -1000, 0.025
    #model.reactions.NH4tpp.bounds = 19.6, 1000
    #model.reactions.GLUt2rpp.bounds = -1000, 0.000512
    #model.reactions.PGCD.bounds = 18.6, 1000
    #model.reactions.TPI.bounds = 9.9,1000
    #model.reactions.RPI.bounds = -1000, -0.073
    #model.reactions.VPAMTr.bounds = -1000, 0.042
    
    # From FVA (Victor): 
    #model.reactions.SERAT.bounds = 9.6, 1000
    #model.reactions.GHMT2r.bounds = 9.45, 1000
    #model.reactions.LSERDHr.bounds = 14.5, 1000
    #model.reactions.SERD_L.bounds = 14.5, 1000
    model.reactions.SERtpp.bounds = 18.5,1000   # *** L-serine export via facilitated transport, value 18.5 from FSEOF
    #model.reactions.TRPS2 .bounds = 14.5, 1000 
    #model.reactions.GHMT2r.bounds = 9.45, 1000
    
    # More (from reactions on metabolite ser__L_p):
    model.reactions.SERt4pp.knock_out() # *** L-serine via sodium symport (periplasm) (uptake)
                                        # Either knocked out or reversed (export of L-serine) - if possible
    #model.reactions.SERt4pp.bounds = -1,-1 # Negative flux increases serine exchange (also increases biomass)
    model.reactions.SERt2rpp.bounds = 0,0 # L-serine reversible transport via proton symport (periplasm) (uptake)
    #model.reactions.PSP_Lpp.bounds = 20,1000
    #model.reactions.SERtex.bounds = -18.5,-18.5 # diffusion, flux cannot be altered in reality
    solution = model.optimize()
    
    print('biomass: ')
    print(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    print('glucose: ')
    print(solution.fluxes.EX_glc__D_e)
    print('serine efflux: ')
    print(solution.fluxes.EX_ser__L_e)
    print("Yield on glucose: ")
    print(solution.fluxes.EX_ser__L_e / (-1*model.reactions.EX_glc__D_e.flux))


WT
biomass: 
0.8769972144269698
glucose: 
-10.0
serine efflux: 
0.0
Yield on glucose: 
0.0
-------------------------------------------------

-------------------------------------------------
ALTERED
biomass: 
0.10122461050799758
glucose: 
-10.0
serine efflux: 
18.5
Yield on glucose: 
1.85


In [43]:
# IN CONCLUSION

with model:
    print("ALTERED")
    model.reactions.SERtpp.bounds = 18.5,1000   # L-serine export via facilitated transport
    
    model.reactions.SERt4pp.knock_out() # L-serine via sodium symport (periplasm) (uptake)
                                        # Either knocked out or reversed (export of L-serine) - if possible
    #model.reactions.SERt4pp.bounds = -1,-1 # Negative flux increases serine exchange (also decreases biomass)
    
    model.reactions.SERt2rpp.knock_out() # L-serine reversible transport via proton symport (periplasm) (uptake)
                                         # Either knocked out or reversed (export of L-serine) - if possible
    #model.reactions.SERt2rpp.bounds = -1,-1 
    
    solution = model.optimize()
    print('Biomass: ')
    print(solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
    print('Glucose: ')
    print(solution.fluxes.EX_glc__D_e)
    print('Serine efflux: ')
    print(solution.fluxes.EX_ser__L_e)
    print("Yield on glucose: ")
    print(solution.fluxes.EX_ser__L_e / (-1*model.reactions.EX_glc__D_e.flux))

ALTERED
Biomass: 
0.10122461050797521
Glucose: 
-10.0
Serine efflux: 
18.5
Yield on glucose: 
1.85



From the reactions investigated:

Reaction SERtpp (export of L-serine) determines the serine efflux (around 21 the cell stops growing completely). 
This is only possible when the uptake reactions SERt4pp and SERt2rpp are either knocked out or reversed (have negative flux values, so that the symport transport goes out of the cell (become export) instead of into the cell - this is probably not physiologically possible though). In the latter cases (negative fluxes), the serine production will increase even further (and the biomass decrease). 